## Install dependencies

In [18]:
# we have to enter OpenAI key here and then we can run all cells freely...
import os
from getpass import getpass
from openai import OpenAI

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

In [19]:
# Homework: Agents
# In this homework, we will learn more about function calling, and we will also explore MCP - model-context protocol.

# Preparation
# First, we'll define a function that we will use when building our agent.

# It will generate fake weather data:

import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:    
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

# def get_weather(*args, **kwargs):
#     print("DEBUG:", *args, *[f"{k}={v}" for k, v in kwargs.items()])


## Q1. Define function description

In [20]:
# Q1. Define function description
# We want to use it as a tool for our agent, so we need to describe it

# How should the description for this function look like? Fill in missing parts

get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "get weather data",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "get weather data from known city and generate for unknown"
            }
        },
        #"required": ["query"],
        "additionalProperties": False
    }
}

# What did you put in TODO3? "city"


In [21]:
# Testing it (Optional)
# If you have OpenAI API Key (or alternative provider), let's test it.

# A question could be "What's the weather like in Germany?"

# Experiment with different system prompts to have better answers from the system.

# You can use chat_assistant.py or implement everything yourself

#!wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py

#!pip install markdown --q

In [22]:
from IPython.display import display, HTML
import markdown # pip install markdown

In [23]:
# connecting to LLM, we entered our API KEY already at the first cell
client = OpenAI()

In [24]:
# Let's use it:

import chat_assistant

tools = chat_assistant.Tools()
tools.add_tool(get_weather, get_weather_tool)

tools.get_tools()

developer_prompt = """
You're a weather bureau assistant. 
You're given a question about current weather conditions in particular city or country and your task is to answer it.

Use weather tool to answer the question. Parse the user question and pass only the name of the city to the tool.
If you unsure what is the city user asked for, double-check exact city name.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)


In [25]:
# and run
# type stop to exit

chat.run()

You: STOP


Chat ended.


## Q2. Adding another tool

In [36]:
# Q2. Adding another tool
# Let's add another tool - a function that can add weather data to our database:

def set_weather(city: str, temp: str) -> None:
    city = city.strip().lower()
    temp = float(temp.strip().lower())
    known_weather_data[city] = temp
    return 'OK'



In [37]:
# Now let's write a description for it.

# What did you write?

# Optionally, you can test it after adding this function.

set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "set weather data",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "name for a city to set weather data"
            },
            "temp": {
                "type": "string",
                "description": "temperature data for a city"
            },
        },
        "additionalProperties": False
    }
}


In [38]:
# Check if tool is added correctly

tools.add_tool(set_weather, set_weather_tool)

tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'get weather data',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'get weather data from known city and generate for unknown'}},
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'set_weather',
  'description': 'set weather data',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'name for a city to set weather data'},
    'temp': {'type': 'string', 'description': 'temperature data for a city'}},
   'additionalProperties': False}}]

In [39]:
# change the prompt to let ChatGPT use second tool to update weather data

developer_prompt = """
You're a weather bureau assistant. 
You have 2 user scenarios of operation:
1/ If you're given a question about current weather conditions in particular city or country -
use get weather data tool to answer the question. 

2/ If user advises that she/he knows the weather in the city and would like to set it up  -
use set weather data tool to set the weather data in the particular city. 

In each scenario parse the user question and pass only the name of the city to the tool.
If you unsure what is the city user asked for, double-check exact city name.

At the end of each response, ask the user if she/he is satisfied and if answer is yes - 
advise to type STOP to end the conversation.
""".strip()

In [40]:
# refresh our chat objects to accomodate new tool

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)


In [41]:
# and run it - stop to exit

chat.run()

You: What is the weather in Berlin now?


You: Thanks! And what is the weather in Tokio?


You: I think it is incorrect - please set the weather for Tokyo as 35


You: Thanks! What is the weather in Tokyo?


You: STop


Chat ended.


In [42]:
known_weather_data

{'berlin': 20.0, 'tokyo': 35.0}

## Q3. Install FastMCP

In [44]:
# MCP
# MCP stands for Model-Context Protocol. It allows LLMs communicate with different tools (like Qdrant). It's function calling, but one step further:

# A tool can export a list of functions it has
# When we include the tool to our Agent, we just need to include the link to the MCP server
# Q3. Install FastMCP
# Let's install a library for MCP - FastMCP:

!pip install fastmcp --q # no long printouts...


In [46]:
# What's the version of FastMCP you installed?
import importlib.metadata

print(importlib.metadata.version("fastmcp"))
# 2.12.2

2.12.2


## Q4. Simple MCP Server

In [50]:
# Q4. Simple MCP Server
# A simple MCP server from the documentation looks like that:

# weather_server.py
# from fastmcp import FastMCP

# mcp = FastMCP("Demo 🚀")

# @mcp.tool
# def add(a: int, b: int) -> int:
#     """Add two numbers"""
#     return a + b


# mcp.run()

In [51]:
# In our case, we need to write docstrings for our functions.

# Let's ask ChatGPT for help:

def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)


def set_weather(city: str, temp: float) -> None:
    """
    Sets the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'



In [ ]:
# Let's change the example for our case and run it

# What do you see in the output?

# Look for a string that matches this template:

# Starting MCP server 'Demo 🚀' with transport '<TODO>'
# What do you have instead of <TODO>?

